In [24]:
import math
import numpy as np
import pandas
import itertools
from sklearn.metrics import precision_score


uii_data = pandas.read_csv('../working_data/updrsii_all_q_p_s.csv')

In [25]:
def query_builder(qs):
    output = ""
    for q in qs:
        add = f'{q} > 0.0 and '
        output = output + add        
    return output[:-5]

def test_q_subset(source,qs,results):
    df_tmp = source.query(query_builder(qs))
    row,_ = df_tmp.shape
    if row > 5:
        y = df_tmp['SIGFALL']
        predictions = [1.0] * len(y)
        p = precision_score(y, predictions, zero_division=0)
        if p > 0.4:
            results.append((qs, p))
       
    
def specific_query_builder(qs):
    output = ""
    for q in qs:
        add = f'{q} > 1.0 and '
        output = output + add        
    return output[:-5]

def specific_test_q_subset(source,qs,results):
    df_tmp = source.query(specific_query_builder(qs))
    row,_ = df_tmp.shape
    if row > 5:
        y = df_tmp['SIGFALL']
        predictions = [1.0] * len(y)
        p = precision_score(y, predictions, zero_division=0)
        if p > 0.1:
            results.append((qs, p))

In [26]:
# Perebor it:

uii_all_qs = {'D_NP2SPCH', 'D_NP2SALV', 'D_NP2SWAL', 'D_NP2EAT',
       'D_NP2DRES', 'D_NP2HYGN', 'D_NP2HWRT', 'D_NP2HOBB', 'D_NP2TURN',
       'D_NP2TRMR', 'D_NP2RISE', 'D_NP2WALK', 'D_NP2FREZ', 'D_NP2PTOT'}

uii_sets = []

for x in range(1,len(uii_all_qs)):
    uii_sets.append(list(itertools.combinations(uii_all_qs, x)))

In [27]:
uii_flatten = [element for sublist in uii_sets for element in sublist]
uii_flatten[0]
len(uii_flatten)

16382

In [28]:
# Try ALL the combinations:

uii_results = []
for qs in uii_flatten:
    test_q_subset(uii_data, qs, uii_results)

In [29]:
uii_results

[(('D_NP2TRMR', 'D_NP2HOBB', 'D_NP2FREZ'), 0.42857142857142855),
 (('D_NP2HWRT', 'D_NP2SWAL', 'D_NP2FREZ'), 0.5),
 (('D_NP2HWRT', 'D_NP2FREZ', 'D_NP2EAT'), 0.5),
 (('D_NP2DRES', 'D_NP2FREZ', 'D_NP2EAT'), 0.45454545454545453),
 (('D_NP2SWAL', 'D_NP2WALK', 'D_NP2FREZ'), 0.42857142857142855),
 (('D_NP2SWAL', 'D_NP2HOBB', 'D_NP2FREZ'), 0.4444444444444444),
 (('D_NP2RISE', 'D_NP2FREZ', 'D_NP2EAT'), 0.4444444444444444),
 (('D_NP2TRMR', 'D_NP2RISE', 'D_NP2WALK', 'D_NP2FREZ'), 0.5),
 (('D_NP2TRMR', 'D_NP2PTOT', 'D_NP2HOBB', 'D_NP2FREZ'), 0.42857142857142855),
 (('D_NP2HWRT', 'D_NP2SWAL', 'D_NP2PTOT', 'D_NP2FREZ'), 0.4444444444444444),
 (('D_NP2HWRT', 'D_NP2SWAL', 'D_NP2HOBB', 'D_NP2FREZ'), 0.6666666666666666),
 (('D_NP2HWRT', 'D_NP2PTOT', 'D_NP2FREZ', 'D_NP2EAT'), 0.5),
 (('D_NP2DRES', 'D_NP2SWAL', 'D_NP2RISE', 'D_NP2EAT'), 0.42857142857142855),
 (('D_NP2DRES', 'D_NP2SWAL', 'D_NP2FREZ', 'D_NP2EAT'), 0.42857142857142855),
 (('D_NP2DRES', 'D_NP2RISE', 'D_NP2FREZ', 'D_NP2EAT'), 0.4285714285714285

In [16]:
# Specfifc

uii_spec_qs = {'D_NP2RISE','D_NP2WALK', 'D_NP2FREZ', 'D_NP2PTOT'}

uii_spec_sets = []
for x in range(1,len(uii_all_qs)):
    uii_spec_sets.append(list(itertools.combinations(uii_spec_qs, x)))
    
uii_spec_flatten = [element for sublist in uii_spec_sets for element in sublist]
uii_spec_flatten[0]
print('count : ',len(uii_spec_flatten))

uii_spec_results = []
for qs in uii_spec_flatten:
    specific_test_q_subset(uii_data, qs, uii_spec_results)
    
uii_spec_results

count :  15


[(('D_NP2PTOT',), 0.16379310344827586),
 (('D_NP2FREZ',), 0.2777777777777778),
 (('D_NP2RISE',), 0.2777777777777778),
 (('D_NP2WALK',), 0.25),
 (('D_NP2PTOT', 'D_NP2FREZ'), 0.29411764705882354),
 (('D_NP2PTOT', 'D_NP2RISE'), 0.3333333333333333),
 (('D_NP2PTOT', 'D_NP2WALK'), 0.2857142857142857),
 (('D_NP2FREZ', 'D_NP2WALK'), 0.2857142857142857),
 (('D_NP2PTOT', 'D_NP2FREZ', 'D_NP2WALK'), 0.2857142857142857)]

In [18]:
print(uii_data.query("SIGFALL == 0").shape)
print(uii_data.query("SIGFALL > 0").shape)

(644, 16)
(82, 16)


In [23]:
82/644

0.12732919254658384

In [22]:
(644 - 82)/644

0.8726708074534162